In [1]:
import requests
import pandas as pd
import pickle
from features.health_insurance import HealthInsurance
import json

In [27]:
df_raw = pd.read_csv('/home/ezequiel/Documentos/Comunidade_DS/health_insurance_cross_sell/data/raw/train.csv')

In [21]:
df_raw.shape

(381109, 12)

In [3]:
df_raw.sample(500).to_csv('./data/df_test.csv', index=False)

In [28]:
model = pickle.load(open('/home/ezequiel/Documentos/Comunidade_DS/health_insurance_cross_sell/models/linear_regression_model.pkl', 'rb'))

In [29]:
pipeline = HealthInsurance()

In [30]:
df_test = df_raw.copy()

In [31]:
df1 = HealthInsurance.columns_rename(pipeline, data=df_test)

In [32]:
df2 = HealthInsurance.feature_engineering(pipeline, data=df1)

In [33]:
df3 = HealthInsurance.data_preparation(pipeline, data=df2)

In [34]:
df_response = HealthInsurance.get_prediction(pipeline, model=model, original_data=df_raw, test_data=df3)

In [36]:
df_response

In [48]:
pd.DataFrame(json.loads(df_response)).head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,score
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1,0.302348
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0,0.035963
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1,0.300988
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0,0.000628
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0,0.000569


In [75]:
df_test = pd.read_csv('./data/df_test.csv')

In [76]:
#data = json.dumps(df_test.sample(500).to_dict(orient='records'))
data = df_test.sample(500).to_json(orient='records')

In [112]:
# API call
#url = 'http://localhost:5000/src/models'
url = 'https://health-insurance-web-app.onrender.com/predict'
header = {'Content-type': 'application/json'}

r = requests.post(url, json=data, headers=header)
#r = requests.post(url=url, json=data)
print(f'Status Code {r.status_code}')

Status Code 502


In [78]:
pd.json_normalize(r.json()).head(20).sort_values('score', ascending=False)

,id,gender,age,driving_license,region_code,previously_insured,vehicle_age,vehicle_damage,annual_premium,policy_sales_channel,vintage,response,score
1,76475,0.138038,0.200000,1,0.187564,0,below_1_year,1,0.960551,61.0,0.591696,0,0.347035
2,135198,0.138038,0.707692,1,0.187564,0,between_1_2_years,1,-1.621992,157.0,0.820069,0,0.317628
7,247562,0.138038,0.430769,1,0.187564,0,between_1_2_years,1,0.980524,124.0,0.169550,1,0.303761
3,130865,0.104363,0.369231,1,0.187564,0,between_1_2_years,1,1.874912,26.0,0.276817,0,0.279339
6,247983,0.138038,0.569231,1,0.167440,0,between_1_2_years,1,0.334061,124.0,0.242215,0,0.278163
19,57885,0.104363,0.076923,1,0.126802,0,below_1_year,1,0.359318,26.0,0.086505,0,0.235069
18,92504,0.138038,0.584615,1,0.098092,0,between_1_2_years,1,0.763604,124.0,0.103806,0,0.219381
14,32869,0.104363,0.076923,1,0.078768,0,below_1_year,1,0.218169,160.0,0.671280,0,0.217398
11,27441,0.104363,0.307692,1,0.070596,0,between_1_2_years,1,-0.276752,124.0,0.647059,1,0.185897
15,169296,0.138038,0.046154,1,0.062634,0,below_1_year,0,-0.343814,160.0,0.802768,0,0.035330


In [2]:
test1 = [{"id":225916,"Gender":"Male","Age":69,"Driving_License":1,"Region_Code":28,"Previously_Insured":0,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"Yes","Annual_Premium":34404,"Policy_Sales_Channel":124,"Vintage":30,"Response":0},{"id":108197,"Gender":"Male","Age":60,"Driving_License":1,"Region_Code":15,"Previously_Insured":1,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"No","Annual_Premium":2630,"Policy_Sales_Channel":160,"Vintage":65,"Response":0},{"id":92008,"Gender":"Male","Age":41,"Driving_License":1,"Region_Code":3,"Previously_Insured":0,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"Yes","Annual_Premium":73221,"Policy_Sales_Channel":26,"Vintage":289,"Response":0},{"id":191052,"Gender":"Male","Age":27,"Driving_License":1,"Region_Code":9,"Previously_Insured":0,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"Yes","Annual_Premium":2630,"Policy_Sales_Channel":156,"Vintage":181,"Response":0},{"id":187752,"Gender":"Male","Age":41,"Driving_License":1,"Region_Code":28,"Previously_Insured":1,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"No","Annual_Premium":30467,"Policy_Sales_Channel":13,"Vintage":270,"Response":0},{"id":216231,"Gender":"Female","Age":41,"Driving_License":1,"Region_Code":28,"Previously_Insured":0,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"Yes","Annual_Premium":57802,"Policy_Sales_Channel":124,"Vintage":172,"Response":0},{"id":315908,"Gender":"Male","Age":68,"Driving_License":1,"Region_Code":8,"Previously_Insured":1,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"No","Annual_Premium":72179,"Policy_Sales_Channel":26,"Vintage":274,"Response":0},{"id":172082,"Gender":"Female","Age":47,"Driving_License":1,"Region_Code":28,"Previously_Insured":1,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"No","Annual_Premium":45620,"Policy_Sales_Channel":122,"Vintage":211,"Response":0},{"id":58502,"Gender":"Male","Age":44,"Driving_License":1,"Region_Code":7,"Previously_Insured":0,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"Yes","Annual_Premium":40947,"Policy_Sales_Channel":124,"Vintage":98,"Response":0},{"id":244356,"Gender":"Male","Age":63,"Driving_License":1,"Region_Code":8,"Previously_Insured":0,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"Yes","Annual_Premium":42146,"Policy_Sales_Channel":26,"Vintage":16,"Response":0}]

In [102]:
data

'[{"id":234405,"Gender":"Male","Age":23,"Driving_License":1,"Region_Code":12.0,"Previously_Insured":1,"Vehicle_Age":"< 1 Year","Vehicle_Damage":"No","Annual_Premium":37993.0,"Policy_Sales_Channel":152.0,"Vintage":247,"Response":0},{"id":76475,"Gender":"Male","Age":33,"Driving_License":1,"Region_Code":28.0,"Previously_Insured":0,"Vehicle_Age":"< 1 Year","Vehicle_Damage":"Yes","Annual_Premium":47109.0,"Policy_Sales_Channel":61.0,"Vintage":181,"Response":0},{"id":135198,"Gender":"Male","Age":66,"Driving_License":1,"Region_Code":28.0,"Previously_Insured":0,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"Yes","Annual_Premium":2630.0,"Policy_Sales_Channel":157.0,"Vintage":247,"Response":0},{"id":130865,"Gender":"Female","Age":44,"Driving_License":1,"Region_Code":28.0,"Previously_Insured":0,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"Yes","Annual_Premium":62857.0,"Policy_Sales_Channel":26.0,"Vintage":90,"Response":0},{"id":89191,"Gender":"Female","Age":48,"Driving_License":1,"Region_Code":30.0,"Pre

In [3]:
test1

[{'id': 225916,
  'Gender': 'Male',
  'Age': 69,
  'Driving_License': 1,
  'Region_Code': 28,
  'Previously_Insured': 0,
  'Vehicle_Age': '1-2 Year',
  'Vehicle_Damage': 'Yes',
  'Annual_Premium': 34404,
  'Policy_Sales_Channel': 124,
  'Vintage': 30,
  'Response': 0},
 {'id': 108197,
  'Gender': 'Male',
  'Age': 60,
  'Driving_License': 1,
  'Region_Code': 15,
  'Previously_Insured': 1,
  'Vehicle_Age': '1-2 Year',
  'Vehicle_Damage': 'No',
  'Annual_Premium': 2630,
  'Policy_Sales_Channel': 160,
  'Vintage': 65,
  'Response': 0},
 {'id': 92008,
  'Gender': 'Male',
  'Age': 41,
  'Driving_License': 1,
  'Region_Code': 3,
  'Previously_Insured': 0,
  'Vehicle_Age': '1-2 Year',
  'Vehicle_Damage': 'Yes',
  'Annual_Premium': 73221,
  'Policy_Sales_Channel': 26,
  'Vintage': 289,
  'Response': 0},
 {'id': 191052,
  'Gender': 'Male',
  'Age': 27,
  'Driving_License': 1,
  'Region_Code': 9,
  'Previously_Insured': 0,
  'Vehicle_Age': '1-2 Year',
  'Vehicle_Damage': 'Yes',
  'Annual_Premium'

In [4]:
test1 = json.dumps(test1)

In [5]:
test1

'[{"id": 225916, "Gender": "Male", "Age": 69, "Driving_License": 1, "Region_Code": 28, "Previously_Insured": 0, "Vehicle_Age": "1-2 Year", "Vehicle_Damage": "Yes", "Annual_Premium": 34404, "Policy_Sales_Channel": 124, "Vintage": 30, "Response": 0}, {"id": 108197, "Gender": "Male", "Age": 60, "Driving_License": 1, "Region_Code": 15, "Previously_Insured": 1, "Vehicle_Age": "1-2 Year", "Vehicle_Damage": "No", "Annual_Premium": 2630, "Policy_Sales_Channel": 160, "Vintage": 65, "Response": 0}, {"id": 92008, "Gender": "Male", "Age": 41, "Driving_License": 1, "Region_Code": 3, "Previously_Insured": 0, "Vehicle_Age": "1-2 Year", "Vehicle_Damage": "Yes", "Annual_Premium": 73221, "Policy_Sales_Channel": 26, "Vintage": 289, "Response": 0}, {"id": 191052, "Gender": "Male", "Age": 27, "Driving_License": 1, "Region_Code": 9, "Previously_Insured": 0, "Vehicle_Age": "1-2 Year", "Vehicle_Damage": "Yes", "Annual_Premium": 2630, "Policy_Sales_Channel": 156, "Vintage": 181, "Response": 0}, {"id": 187752, 

In [21]:
url = 'https://health-insurance-web-app.onrender.com/predict'
header = {'Content-type': 'application/json'}

r = requests.post(url, json=test2, headers=header)
#r = requests.post(url=url, data=test1)
print(f'Status Code {r.status_code}')

Status Code 200


requests.models.Response

In [8]:
test2 = [{"id":225916,"Gender":"Male","Age":69,"Driving_License":1,"Region_Code":28,"Previously_Insured":0,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"Yes","Annual_Premium":34404,"Policy_Sales_Channel":124,"Vintage":30,"Response":0},{"id":108197,"Gender":"Male","Age":60,"Driving_License":1,"Region_Code":15,"Previously_Insured":1,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"No","Annual_Premium":2630,"Policy_Sales_Channel":160,"Vintage":65,"Response":0},{"id":92008,"Gender":"Male","Age":41,"Driving_License":1,"Region_Code":3,"Previously_Insured":0,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"Yes","Annual_Premium":73221,"Policy_Sales_Channel":26,"Vintage":289,"Response":0},{"id":191052,"Gender":"Male","Age":27,"Driving_License":1,"Region_Code":9,"Previously_Insured":0,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"Yes","Annual_Premium":2630,"Policy_Sales_Channel":156,"Vintage":181,"Response":0},{"id":187752,"Gender":"Male","Age":41,"Driving_License":1,"Region_Code":28,"Previously_Insured":1,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"No","Annual_Premium":30467,"Policy_Sales_Channel":13,"Vintage":270,"Response":0},{"id":216231,"Gender":"Female","Age":41,"Driving_License":1,"Region_Code":28,"Previously_Insured":0,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"Yes","Annual_Premium":57802,"Policy_Sales_Channel":124,"Vintage":172,"Response":0},{"id":315908,"Gender":"Male","Age":68,"Driving_License":1,"Region_Code":8,"Previously_Insured":1,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"No","Annual_Premium":72179,"Policy_Sales_Channel":26,"Vintage":274,"Response":0},{"id":172082,"Gender":"Female","Age":47,"Driving_License":1,"Region_Code":28,"Previously_Insured":1,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"No","Annual_Premium":45620,"Policy_Sales_Channel":122,"Vintage":211,"Response":0},{"id":58502,"Gender":"Male","Age":44,"Driving_License":1,"Region_Code":7,"Previously_Insured":0,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"Yes","Annual_Premium":40947,"Policy_Sales_Channel":124,"Vintage":98,"Response":0},{"id":244356,"Gender":"Male","Age":63,"Driving_License":1,"Region_Code":8,"Previously_Insured":0,"Vehicle_Age":"1-2 Year","Vehicle_Damage":"Yes","Annual_Premium":42146,"Policy_Sales_Channel":26,"Vintage":16,"Response":0}]

In [15]:
test2

'[{"id": 225916, "Gender": "Male", "Age": 69, "Driving_License": 1, "Region_Code": 28, "Previously_Insured": 0, "Vehicle_Age": "1-2 Year", "Vehicle_Damage": "Yes", "Annual_Premium": 34404, "Policy_Sales_Channel": 124, "Vintage": 30, "Response": 0}, {"id": 108197, "Gender": "Male", "Age": 60, "Driving_License": 1, "Region_Code": 15, "Previously_Insured": 1, "Vehicle_Age": "1-2 Year", "Vehicle_Damage": "No", "Annual_Premium": 2630, "Policy_Sales_Channel": 160, "Vintage": 65, "Response": 0}, {"id": 92008, "Gender": "Male", "Age": 41, "Driving_License": 1, "Region_Code": 3, "Previously_Insured": 0, "Vehicle_Age": "1-2 Year", "Vehicle_Damage": "Yes", "Annual_Premium": 73221, "Policy_Sales_Channel": 26, "Vintage": 289, "Response": 0}, {"id": 191052, "Gender": "Male", "Age": 27, "Driving_License": 1, "Region_Code": 9, "Previously_Insured": 0, "Vehicle_Age": "1-2 Year", "Vehicle_Damage": "Yes", "Annual_Premium": 2630, "Policy_Sales_Channel": 156, "Vintage": 181, "Response": 0}, {"id": 187752, 

In [11]:
type(test2)

str

In [9]:
test2 = json.dumps(test2)

In [13]:
json.loads(test2)

[{'id': 225916,
  'Gender': 'Male',
  'Age': 69,
  'Driving_License': 1,
  'Region_Code': 28,
  'Previously_Insured': 0,
  'Vehicle_Age': '1-2 Year',
  'Vehicle_Damage': 'Yes',
  'Annual_Premium': 34404,
  'Policy_Sales_Channel': 124,
  'Vintage': 30,
  'Response': 0},
 {'id': 108197,
  'Gender': 'Male',
  'Age': 60,
  'Driving_License': 1,
  'Region_Code': 15,
  'Previously_Insured': 1,
  'Vehicle_Age': '1-2 Year',
  'Vehicle_Damage': 'No',
  'Annual_Premium': 2630,
  'Policy_Sales_Channel': 160,
  'Vintage': 65,
  'Response': 0},
 {'id': 92008,
  'Gender': 'Male',
  'Age': 41,
  'Driving_License': 1,
  'Region_Code': 3,
  'Previously_Insured': 0,
  'Vehicle_Age': '1-2 Year',
  'Vehicle_Damage': 'Yes',
  'Annual_Premium': 73221,
  'Policy_Sales_Channel': 26,
  'Vintage': 289,
  'Response': 0},
 {'id': 191052,
  'Gender': 'Male',
  'Age': 27,
  'Driving_License': 1,
  'Region_Code': 9,
  'Previously_Insured': 0,
  'Vehicle_Age': '1-2 Year',
  'Vehicle_Damage': 'Yes',
  'Annual_Premium'

In [20]:
pd.json_normalize(test2)

NotImplementedError: 